<a href="https://colab.research.google.com/github/EdWangward/khclassifier/blob/main/kimharrisonclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

folder = '/content/sample_data/train/'
photos, labels = list(), list()

for file in listdir(folder):
  output = 0
  if file.startswith('Kim'):
    output = 1
  photo = load_img(folder + file, target_size=(80,80))
  photo = img_to_array(photo)
  photos.append(photo)
  labels.append(output)
photos = asarray(photos)
labels = asarray(labels)

save('dogcatsp.npy',photos)
save('dogcatsl.npy', labels)
print(labels)

In [ ]:
from random import seed
from random import random
from shutil import copyfile
from os import listdir
from os import makedirs
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
  labeldirs = ['Harrison/', 'Kim/']
  for labeldir in labeldirs:
    newdir = dataset_home + subdir + labeldir
    makedirs(newdir, exist_ok=True)

In [ ]:
seed(1)
val_ratio = 0.25

src_directory = '/content/sample_data/train/'
for file in listdir(src_directory):
  src = src_directory + '/' + file
  dst_dir = 'train/'
  if random()< val_ratio:
    dst_dir = 'test/'
  if file.startswith('Kim'):
    dst = dataset_home + dst_dir + 'Kim/' + file
    copyfile(src, dst)
  elif file.startswith('Harrison'):
    dst = dataset_home + dst_dir +'Harrison/'+ file
    copyfile(src, dst)


In [ ]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
 
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(80, 80, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(rescale=1.0/160)
	# specify imagenet mean values for centering
	# prepare iterator
	train_it = datagen.flow_from_directory('/content/dataset_dogs_vs_cats/train/',
		class_mode='binary', batch_size=64, target_size=(80, 80))
	test_it = datagen.flow_from_directory('/content/dataset_dogs_vs_cats/test/',
		class_mode='binary', batch_size=64, target_size=(80, 80))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=1)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	model.save('final_model.h5')
 
# entry point, run the test harness
run_test_harness()


In [ ]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import PIL


# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(80, 80))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 80, 80, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [40, 40, 40]
	return img
 
# load an image and predict the class

def run_example():
  img = load_image('/content/sample_img21.png')
  model = load_model('final_model.h5')
  result = model.predict(img)
  if result[0] == 1:
    print("Kim")
  else:
    print("Harrison")

run_example()